# Create platemap figures based on the platemap files

## Import libraries

In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(platetools))

Warning message:
“package ‘ggplot2’ was built under R version 4.2.3”
Warning message:
“package ‘platetools’ was built under R version 4.2.3”


# Set paths to each platemap and output

In [2]:
platemap_files <- list.files(pattern = "_platemap\\.csv$", full.names = TRUE)
print(platemap_files)

output_fig_dir <- file.path("platemap_figures")
if (!dir.exists(output_fig_dir)) {
    dir.create(output_fig_dir)
}
platemap_suffix <- "_platemap_figure.png"

# Define output figure paths
output_platemap_files <- list()
for (platemap_file in platemap_files) {
    # Extract plate name and remove suffix 
    plate <- basename(platemap_file)
    plate <- stringr::str_remove(plate, "_platemap.csv") 
    
    output_platemap_files[[plate]] <- file.path(output_fig_dir, paste0(plate, platemap_suffix))
}

print(output_platemap_files)


[1] "./dmso_training_platemap.csv"
$dmso_training
[1] "platemap_figures/dmso_training_platemap_figure.png"



## Load in each platemap CSV for each plate

In [3]:
# Load in all platemap CSV files
platemap_dfs <- list()
for (plate in names(output_platemap_files)) {
    # Find the umap file associated with the plate
    platemap_file <- platemap_files[stringr::str_detect(platemap_files, plate)]
    
    # Load in the umap data
    df <- readr::read_csv(
    platemap_file,
    col_types = readr::cols(.default = "c")
)

    platemap_dfs[[plate]] <- df 
}

print(platemap_dfs)

$dmso_training
# A tibble: 56 × 7
   WellRow WellCol well_position heart_number treatment cell_type
   <chr>   <chr>   <chr>         <chr>        <chr>     <chr>    
 1 B       2       B02           2            DMSO      Healthy  
 2 B       3       B03           7            DMSO      Healthy  
 3 B       4       B04           2            None      Healthy  
 4 B       5       B05           23           DMSO      Failing  
 5 B       6       B06           25           DMSO      Failing  
 6 B       7       B07           46           DMSO      Failing  
 7 B       8       B08           47           DMSO      Failing  
 8 B       9       B09           2            DMSO      Healthy  
 9 B       10      B10           7            DMSO      Healthy  
10 B       11      B11           2            None      Healthy  
# ℹ 46 more rows
# ℹ 1 more variable: heart_failure_type <chr>



## Create platemap figure for only plate 4

In [4]:
for (plate in names(platemap_dfs)) {
 {
    # output for each plate
    output_file <- output_platemap_files[[plate]]
    output_file <- paste0(output_file)
    
    platemap <-
        platetools::raw_map(
            data = platemap_dfs[[plate]]$heart_number,
            well = platemap_dfs[[plate]]$well_position,
            plate = 96,
            size = 8
        ) +
        ggtitle(paste("Platemap layout for plate", plate)) +
        theme(plot.title = element_text(hjust=0.5, size = 10, face = "bold", margin = margin(b = -5))) +
        ggplot2::geom_point(aes(shape = platemap_dfs[[plate]]$cell_type)) +
        ggplot2::scale_shape_discrete(name = "Cell Type") +
        ggplot2::scale_fill_discrete(name = "Heart Number") +
        theme(
            legend.position = "right",
            legend.title = element_text(size = 9),
            legend.text = element_text(size = 8),
        ) +
        guides(
        shape = guide_legend(override.aes = list(size = 2)),
        fill = guide_legend(override.aes = list(size = 5))
    )

    ggsave(
        output_file,
        platemap,
        dpi = 500,
        height = 3.5,
        width = 6
    )
    }
}